In [ ]:
!pip install torch transformers peft datasets scikit-learn matplotlib seaborn textblob pandas

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, precision_recall_fscore_support, accuracy_score
)
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
import warnings
warnings.filterwarnings('ignore')

try:
    from peft import LoraConfig, get_peft_model, TaskType
    PEFT_AVAILABLE = True
except ImportError:
    print("PEFT not available. Using standard fine-tuning.")
    PEFT_AVAILABLE = False

In [ ]:
# =========================
# EDA Functions
# =========================
def executive_summary(df):
    print("Executive Summary of E-commerce Reviews")
    print(f"Total reviews analyzed: {len(df)}")
    pos = (df['sentiment'] == 'positive').mean()
    neu = (df['sentiment'] == 'neutral').mean()
    neg = (df['sentiment'] == 'negative').mean()
    print(f"Positive reviews: {pos:.1%}")
    print(f"Neutral reviews: {neu:.1%}")
    print(f"Negative reviews: {neg:.1%}")
    aspects = ['aspect_fit', 'aspect_quality', 'aspect_style', 'aspect_comfort', 'aspect_price']
    negative_df = df[df['sentiment'] == 'negative']
    if len(negative_df) > 0:
        top_aspects = negative_df[aspects].sum().sort_values(ascending=False)
        print(f"Top reasons for negative reviews: {list(top_aspects.index.str.replace('aspect_', '').str.title())[:3]}")
    print("\nActionable Insights:")
    print("- Focus on fit and quality issues that drive most negative sentiment")
    print("- Customers appreciate well-fitting, comfortable clothes—highlight these in marketing")
    print("- Addressing sizing and quality complaints can reduce negative reviews significantly\n")

def plot_overall_sentiment(df):
    plt.figure(figsize=(7, 4))
    sns.countplot(data=df, x='sentiment', order=['positive', 'neutral', 'negative'], palette='viridis')
    plt.title("Overall Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Number of Reviews")
    plt.show()

def plot_negative_aspect_breakdown(df):
    aspects = ['aspect_fit', 'aspect_quality', 'aspect_style', 'aspect_comfort', 'aspect_price']
    negative_df = df[df['sentiment'] == 'negative']
    if len(negative_df) > 0:
        aspect_counts = negative_df[aspects].sum().sort_values(ascending=False)
        plt.figure(figsize=(8, 5))
        sns.barplot(x=aspect_counts.index.str.replace('aspect_', '').str.title(),
                    y=aspect_counts.values, palette="mako")
        plt.title("Top Aspects in Negative Reviews")
        plt.xlabel("Aspect")
        plt.ylabel("Count")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

def plot_aspect_sentiment_heatmap(df):
    aspects = ['aspect_fit', 'aspect_quality', 'aspect_style', 'aspect_comfort', 'aspect_price']
    aspect_sentiment = {}
    for aspect in aspects:
        aspect_sentiment[aspect] = df.groupby('sentiment')[aspect].sum()
    aspect_sentiment_df = pd.DataFrame(aspect_sentiment)
    aspect_sentiment_df = aspect_sentiment_df.rename(columns=lambda x: x.replace('aspect_', '').title())
    plt.figure(figsize=(9, 4))
    sns.heatmap(aspect_sentiment_df.T, annot=True, fmt='d', cmap='coolwarm')
    plt.title('Mentions of Each Aspect per Sentiment')
    plt.xlabel('Sentiment')
    plt.ylabel('Aspect')
    plt.tight_layout()
    plt.show()


In [ ]:
# =========================
# Custom Dataset Class
# =========================

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# =========================
# Business Insights/Aspect Extraction
# =========================

class BusinessInsightExtractor:
    def __init__(self):
        self.aspects = {
            'fit': ['fit', 'fits', 'fitting', 'size', 'sized', 'sizing', 'tight', 'loose', 'small', 'large', 'runs'],
            'quality': ['quality', 'material', 'fabric', 'cotton', 'polyester', 'durable', 'cheap', 'flimsy', 'well made', 'poorly made'],
            'style': ['style', 'design', 'look', 'appearance', 'fashionable', 'trendy', 'color', 'pattern', 'cute', 'ugly'],
            'comfort': ['comfort', 'comfortable', 'soft', 'cozy', 'breathable', 'itchy', 'scratchy', 'harsh'],
            'price': ['price', 'cost', 'expensive', 'cheap', 'affordable', 'value', 'worth', 'money', 'overpriced']
        }
    def extract_opinion_phrases(self, text):
        contrast_pattern = r'\\b(but|however|although|though|yet|while|whereas)\\b'
        parts = re.split(contrast_pattern, text, flags=re.IGNORECASE)
        phrases = []
        for i in range(0, len(parts), 2):
            if i < len(parts):
                phrase = parts[i].strip()
                if phrase:
                    phrases.append(phrase)
        return phrases
    def analyze_aspect_sentiment(self, text):
        text_lower = text.lower()
        aspect_sentiments = {}
        for aspect, keywords in self.aspects.items():
            aspect_mentioned = any(keyword in text_lower for keyword in keywords)
            if aspect_mentioned:
                blob = TextBlob(text)
                sentiment_score = blob.sentiment.polarity
                if sentiment_score > 0.1:
                    sentiment = 'positive'
                elif sentiment_score < -0.1:
                    sentiment = 'negative'
                else:
                    sentiment = 'neutral'
                aspect_sentiments[aspect] = {
                    'sentiment': sentiment,
                    'score': sentiment_score,
                    'mentioned_keywords': [kw for kw in keywords if kw in text_lower]
                }
        return aspect_sentiments

    def extract_mixed_reviews(self, df):
        mixed_reviews = []
        for idx, row in df.iterrows():
            text = row['review_text']
            rating = row['rating']
            if rating >= 4:
                phrases = self.extract_opinion_phrases(text)
                if len(phrases) > 1:
                    aspect_analysis = self.analyze_aspect_sentiment(text)
                    sentiments = [asp['sentiment'] for asp in aspect_analysis.values()]
                    if 'positive' in sentiments and 'negative' in sentiments:
                        mixed_reviews.append({
                            'review_id': idx,
                            'rating': rating,
                            'text': text,
                            'phrases': phrases,
                            'aspect_analysis': aspect_analysis
                        })
        return mixed_reviews

    def generate_business_recommendations(self, df):
        recommendations = {}
        negative_reviews = df[df['rating'] <= 2]
        if len(negative_reviews) == 0:
            return recommendations
        for aspect in self.aspects.keys():
            aspect_issues = []
            aspect_count = 0
            for _, row in negative_reviews.iterrows():
                text = row['review_text']
                aspect_analysis = self.analyze_aspect_sentiment(text)
                if aspect in aspect_analysis and aspect_analysis[aspect]['sentiment'] == 'negative':
                    aspect_count += 1
                    aspect_issues.append({
                        'text': text,
                        'keywords': aspect_analysis[aspect]['mentioned_keywords']
                    })
            if aspect_count > 0:
                recs = self._generate_aspect_recommendations(aspect)
                recommendations[aspect] = {
                    'issue_count': aspect_count,
                    'percentage': (aspect_count / len(negative_reviews)) * 100,
                    'recommendations': recs,
                    'sample_issues': aspect_issues[:3]
                }
        return recommendations
    def _generate_aspect_recommendations(self, aspect):
        recommendations = {
            'fit': [
                "Update sizing charts with detailed measurements",
                "Add customer fit photos and reviews",
                "Implement virtual try-on technology",
                "Offer free returns for sizing issues",
                "Create size comparison guides"
            ],
            'quality': [
                "Review material sourcing and supplier quality",
                "Implement stricter quality control processes",
                "Add detailed material descriptions to product pages",
                "Offer extended warranties",
                "Provide care instructions to extend product life"
            ],
            'style': [
                "Conduct customer style preference surveys",
                "A/B test new designs before full production",
                "Expand color and pattern options",
                "Create seasonal trend reports",
                "Collaborate with fashion influencers for feedback"
            ],
            'comfort': [
                "Test products with focus groups for comfort",
                "Use softer, more breathable materials",
                "Add comfort features (padding, seamless construction)",
                "Provide comfort guarantees",
                "Create comfort-focused product lines"
            ],
            'price': [
                "Conduct competitive pricing analysis",
                "Create value proposition messaging",
                "Offer loyalty discounts and rewards",
                "Bundle products for better value",
                "Implement dynamic pricing strategies"
            ]
        }
        return recommendations.get(aspect, ["Review and improve this aspect based on customer feedback"])


In [ ]:
# =========================
# Aspect Bias Sentimental Analysis LLM Pipeline
# =========================

class ABSALLMPipeline:
    def __init__(self, model_name='distilbert-base-uncased', use_lora=True):
        self.model_name = model_name
        self.use_lora = use_lora and PEFT_AVAILABLE
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        print(f"Fine-tuning method: {'LoRA' if self.use_lora else 'Full Fine-tuning'}")
        self.tokenizer = None
        self.model = None
        self.peft_model = None
        self.label_encoder = LabelEncoder()
        self.trainer = None
        self.insight_extractor = BusinessInsightExtractor()
        if self.use_lora:
            self.lora_config = LoraConfig(
                task_type=TaskType.SEQ_CLS,
                inference_mode=False,
                r=16,
                lora_alpha=32,
                lora_dropout=0.1,
                target_modules=self._get_target_modules(model_name),
                bias="none",
            )
    def _get_target_modules(self, model_name):
        if 'distilbert' in model_name.lower():
            return ["q_lin", "v_lin", "k_lin", "out_lin"]
        elif 'roberta' in model_name.lower() or 'bert' in model_name.lower():
            return ["query", "value", "key", "dense"]
        else:
            return ["query", "value"]
    def load_and_preprocess_data(self):
        print("Loading real dataset...")
        ds = load_dataset("Censius-AI/ECommerce-Women-Clothing-Reviews")
        df = pd.DataFrame(ds["train"])
        df.columns = [col.strip().replace(' ', '_').lower() for col in df.columns]
        df = df.dropna(subset=['review_text'])
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle
        # 80% train, 20% test
        train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['rating'])
        print(f"Loaded {len(train_df)} training and {len(test_df)} testing samples.")
        def create_sentiment_label(rating):
            if rating <= 2:
                return 'negative'
            elif rating == 3:
                return 'neutral'
            else:
                return 'positive'
        train_df['sentiment'] = train_df['rating'].apply(create_sentiment_label)
        test_df['sentiment'] = test_df['rating'].apply(create_sentiment_label)
        return train_df, test_df
    def extract_aspects_and_enhance_text(self, df):
        aspects = self.insight_extractor.aspects
        for aspect, keywords in aspects.items():
            pattern = '|'.join([re.escape(kw) for kw in keywords])
            df[f'aspect_{aspect}'] = df['review_text'].str.lower().str.contains(pattern, na=False, regex=True)
        def enhance_text_with_aspects(row):
            text = row['review_text']
            mentioned_aspects = []
            for aspect in aspects.keys():
                if row[f'aspect_{aspect}']:
                    mentioned_aspects.append(aspect)
            if mentioned_aspects:
                aspect_info = f"[ASPECTS: {', '.join(mentioned_aspects)}] "
                return aspect_info + text
            return text
        df['enhanced_text'] = df.apply(enhance_text_with_aspects, axis=1)
        return df
    def prepare_model_and_tokenizer(self, num_labels):
        print(f"Loading {self.model_name} model and tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name, num_labels=num_labels, ignore_mismatched_sizes=True)
        self.model.resize_token_embeddings(len(self.tokenizer))
        if self.use_lora:
            print("Applying LoRA configuration...")
            self.peft_model = get_peft_model(self.model, self.lora_config)
            self.peft_model.print_trainable_parameters()
            self.peft_model.to(self.device)
            self.model = self.peft_model
        else:
            print("Using full fine-tuning...")
            self.model.to(self.device)
            total_params = sum(p.numel() for p in self.model.parameters())
            trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
            print(f"Total parameters: {total_params:,}")
            print(f"Trainable parameters: {trainable_params:,}")
    def create_datasets(self, texts, labels):
        return ReviewDataset(texts, labels, self.tokenizer)
    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
        accuracy = accuracy_score(labels, predictions)
        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }
    def train_model(self, train_dataset, output_dir='./results'):
        print("Training model...")
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=3 if self.use_lora else 2,
            per_device_train_batch_size=16 if self.use_lora else 8,
            per_device_eval_batch_size=32 if self.use_lora else 16,
            learning_rate=1e-4 if self.use_lora else 2e-5,
            warmup_steps=100,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=50,
            report_to=None,
            dataloader_pin_memory=False,
            fp16=torch.cuda.is_available(),
        )
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            compute_metrics=self.compute_metrics
        )
        self.trainer.train()
        return self.trainer
    def evaluate_model(self, test_dataset, test_labels):
        print("Evaluating model...")
        predictions = self.trainer.predict(test_dataset)
        y_pred = np.argmax(predictions.predictions, axis=1)
        y_true = test_labels
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
        accuracy = accuracy_score(y_true, y_pred)
        print(f"\nTest Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"\nDetailed Classification Report:")
        target_names = self.label_encoder.classes_
        print(classification_report(y_true, y_pred, target_names=target_names))
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'y_true': y_true,
            'y_pred': y_pred,
            'predictions': predictions
        }
    def generate_business_insights(self, df):
        print("\n" + "="*60)
        print("BUSINESS INSIGHTS EXTRACTION")
        print("="*60)
        mixed_reviews = self.insight_extractor.extract_mixed_reviews(df)
        print(f"\nFound {len(mixed_reviews)} mixed sentiment reviews")
        recommendations = self.insight_extractor.generate_business_recommendations(df)
        print("\n" + "="*50)
        print("KEY BUSINESS INSIGHTS")
        print("="*50)
        if mixed_reviews:
            print(f"\n1. MIXED SENTIMENT ANALYSIS:")
            print(f"   Found {len(mixed_reviews)} reviews with mixed sentiments")
            print(f"   These are high-rated reviews that still contain negative feedback")
            for i, review in enumerate(mixed_reviews[:3]):
                print(f"\n   Example {i+1} (Rating: {review['rating']}/5):")
                print(f"   Text: \"{review['text'][:100]}...\"")
                print(f"   Aspects: {list(review['aspect_analysis'].keys())}")
        print(f"\n2. ASPECT-BASED RECOMMENDATIONS:")
        for aspect, data in recommendations.items():
            print(f"\n   {aspect.upper()} Issues:")
            print(f"   - {data['issue_count']} complaints ({data['percentage']:.1f}% of negative reviews)")
            print(f"   - Top Recommendations:")
            for rec in data['recommendations'][:3]:
                print(f"     • {rec}")
        return {
            'mixed_reviews': mixed_reviews,
            'recommendations': recommendations
        }
    def run_pipeline(self):
        print("Starting ABSA Pipeline...")
        train_df, test_df = self.load_and_preprocess_data()
        train_df = self.extract_aspects_and_enhance_text(train_df)
        test_df = self.extract_aspects_and_enhance_text(test_df)

        # EDA on training set
        print("\n[EDA on Training Set]")
        executive_summary(train_df)
        plot_overall_sentiment(train_df)
        plot_negative_aspect_breakdown(train_df)
        plot_aspect_sentiment_heatmap(train_df)

        # Generate business insights from training data
        insights = self.generate_business_insights(train_df)

        # Prepare label encoding and texts
        y_train = self.label_encoder.fit_transform(train_df['sentiment'])
        X_train = train_df['enhanced_text'].values
        y_test = self.label_encoder.transform(test_df['sentiment'])
        X_test = test_df['enhanced_text'].values

        # Prepare model and tokenizer
        self.prepare_model_and_tokenizer(len(self.label_encoder.classes_))

        # Datasets
        train_dataset = self.create_datasets(X_train, y_train)
        test_dataset = self.create_datasets(X_test, y_test)

        # Train model
        self.train_model(train_dataset)

        # Evaluate on test set
        test_results = self.evaluate_model(test_dataset, y_test)

        return {
          'test_results': test_results,
          'business_insights': insights,
          'model': self.model,
          'tokenizer': self.tokenizer,
          'train_data': train_df,
          'test_data': test_df,
          'label_encoder': self.label_encoder
         }


In [ ]:
# =========================
# Model Training, Evaluation & Main
# =========================

if __name__ == "__main__":
    # Instantiate the pipeline (choose model, and LoRA or full FT as desired)
    pipeline = ABSALLMPipeline(
        model_name='distilbert-base-uncased',
        use_lora=True  # or False if you do not want LoRA
    )

    # Run the pipeline end-to-end (this includes data load, EDA, insights, training, evaluation)
    results = pipeline.run_pipeline()

    # Show model performance metrics on test set
    print("\n" + "="*50)
    print("FINAL RESULTS SUMMARY")
    print("="*50)
    print(f"Model Performance on Test Set (20% of the real data):")
    print(f"  Accuracy: {results['test_results']['accuracy']:.4f}")
    print(f"  Precision: {results['test_results']['precision']:.4f}")
    print(f"  Recall: {results['test_results']['recall']:.4f}")
    print(f"  F1-Score: {results['test_results']['f1']:.4f}")

    print("\nBusiness Insights Generated:")
    print(f"  Mixed Reviews Found: {len(results['business_insights']['mixed_reviews'])}")
    print(f"  Aspects with Recommendations: {len(results['business_insights']['recommendations'])}")
    print("\nExample Recommendations:")
    for aspect, data in results['business_insights']['recommendations'].items():
        print(f"- {aspect.title()}: {data['recommendations'][:2]}")

    # Example: Extract and print detailed business insights for a sample review
    extractor = BusinessInsightExtractor()
    sample_review = "The fabric is soft but the sizing is off."
    print("\n--- Example: Deep Opinion Extraction ---")
    print("Review:", sample_review)
    phrases = extractor.extract_opinion_phrases(sample_review)
    for p in phrases:
        asp = extractor.analyze_aspect_sentiment(p)
        print(f"  Phrase: {p}\n  Aspect Sentiment: {asp}")


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_recall_fscore_support, accuracy_score
)
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class ABSALLMPipeline:
    def __init__(self, model_name='distilbert-base-uncased', use_lora=True, lora_config=None):
        """
        Initialize ABSA Pipeline with option for LoRA fine-tuning

        Args:
            model_name: Model to use ('distilbert-base-uncased', 'roberta-base', 'bert-base-uncased')
            use_lora: Whether to use LoRA fine-tuning (True) or full fine-tuning (False)
            lora_config: Custom LoRA configuration
        """
        self.model_name = model_name
        self.use_lora = use_lora
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        print(f"Fine-tuning method: {'LoRA' if use_lora else 'Full Fine-tuning'}")

        self.tokenizer = None
        self.model = None
        self.peft_model = None
        self.label_encoder = LabelEncoder()
        self.trainer = None

        # Default LoRA configuration
        self.lora_config = lora_config or LoraConfig(
            task_type=TaskType.SEQ_CLS,  # Sequence Classification
            inference_mode=False,
            r=16,                        # Rank - higher = more parameters but better performance
            lora_alpha=32,               # LoRA scaling parameter
            lora_dropout=0.1,            # Dropout for LoRA layers
            target_modules=self._get_target_modules(model_name),  # Which layers to adapt
            bias="none",                 # Whether to adapt bias parameters
        )

    def _get_target_modules(self, model_name):
        """Get target modules for LoRA based on model architecture"""
        if 'distilbert' in model_name.lower():
            return ["q_lin", "v_lin", "k_lin", "out_lin"]
        elif 'roberta' in model_name.lower() or 'bert' in model_name.lower():
            return ["query", "value", "key", "dense"]
        else:
            # Default for most transformer models
            return ["query", "value"]

    def load_and_preprocess_data(self):
        """Load and preprocess the dataset"""
        print("Loading dataset...")
        ds = load_dataset("Censius-AI/ECommerce-Women-Clothing-Reviews")
        df = pd.DataFrame(ds["train"])

        print(f"Dataset shape: {df.shape}")

        # Remove rows with missing review text
        df = df.dropna(subset=['Review Text'])

        # Sample data for demonstration (remove this line for full dataset)
        df = df.sample(n=min(15000, len(df)), random_state=42).reset_index(drop=True)
        print(f"Sampled dataset shape: {df.shape}")

        # Create sentiment labels from rating
        def create_sentiment_label(rating):
            if rating <= 2:
                return 'negative'
            elif rating == 3:
                return 'neutral'
            else:
                return 'positive'

        df['sentiment'] = df['Rating'].apply(create_sentiment_label)

        print(f"Sentiment distribution:")
        print(df['sentiment'].value_counts())

        return df

    def extract_aspects_and_enhance_text(self, df):
        """Extract aspects and enhance text with aspect information"""
        print("Extracting aspects and enhancing text...")

        # Define common aspects for clothing reviews
        aspects = {
            'fit': ['fit', 'fits', 'fitting', 'size', 'sized', 'sizing', 'tight', 'loose', 'small', 'large'],
            'quality': ['quality', 'material', 'fabric', 'cotton', 'polyester', 'durable', 'cheap', 'flimsy'],
            'style': ['style', 'design', 'look', 'appearance', 'fashionable', 'trendy', 'color', 'pattern'],
            'comfort': ['comfort', 'comfortable', 'soft', 'cozy', 'breathable', 'itchy', 'scratchy'],
            'price': ['price', 'cost', 'expensive', 'cheap', 'affordable', 'value', 'worth', 'money']
        }

        # Extract aspect mentions
        for aspect, keywords in aspects.items():
            df[f'aspect_{aspect}'] = df['Review Text'].str.lower().str.contains('|'.join(keywords), na=False)

        # Create enhanced text with aspect information
        def enhance_text_with_aspects(row):
            text = row['Review Text']
            mentioned_aspects = []

            for aspect in aspects.keys():
                if row[f'aspect_{aspect}']:
                    mentioned_aspects.append(aspect)

            if mentioned_aspects:
                aspect_info = f"[ASPECTS: {', '.join(mentioned_aspects)}] "
                return aspect_info + text
            return text

        df['enhanced_text'] = df.apply(enhance_text_with_aspects, axis=1)

        return df

    def prepare_model_and_tokenizer(self, num_labels):
        """Initialize tokenizer and model with optional LoRA"""
        print(f"Loading {self.model_name} model and tokenizer...")

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Load base model
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=num_labels
        )

        if self.use_lora:
            print("Applying LoRA configuration...")
            print(f"LoRA Config: {self.lora_config}")

            # Apply LoRA to the model
            self.peft_model = get_peft_model(self.model, self.lora_config)

            # Print trainable parameters
            self.peft_model.print_trainable_parameters()

            # Move to device
            self.peft_model.to(self.device)

            # Use peft_model for training
            self.model = self.peft_model
        else:
            print("Using full fine-tuning...")
            # Move model to device
            self.model.to(self.device)

            total_params = sum(p.numel() for p in self.model.parameters())
            trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
            print(f"Total parameters: {total_params:,}")
            print(f"Trainable parameters: {trainable_params:,}")

    def create_datasets(self, train_texts, train_labels, val_texts, val_labels, test_texts, test_labels):
        """Create PyTorch datasets"""
        train_dataset = ReviewDataset(train_texts, train_labels, self.tokenizer)
        val_dataset = ReviewDataset(val_texts, val_labels, self.tokenizer)
        test_dataset = ReviewDataset(test_texts, test_labels, self.tokenizer)

        return train_dataset, val_dataset, test_dataset

    def compute_metrics(self, eval_pred):
        """Compute metrics for evaluation"""
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
        accuracy = accuracy_score(labels, predictions)

        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

    def train_model(self, train_dataset, val_dataset, output_dir='./results'):
        """Train the model using Hugging Face Trainer"""
        print("Training model...")

        # Adjust training arguments based on fine-tuning method
        if self.use_lora:
            # LoRA settings - can use higher batch sizes and learning rates
            training_args = TrainingArguments(
                output_dir=output_dir,
                num_train_epochs=5,
                per_device_train_batch_size=32,  # Higher batch size for LoRA
                per_device_eval_batch_size=64,
                learning_rate=1e-4,              # Higher learning rate for LoRA
                warmup_steps=100,
                weight_decay=0.01,
                logging_dir='./logs',
                logging_steps=50,
                evaluation_strategy="steps",
                eval_steps=200,
                save_strategy="steps",
                save_steps=200,
                load_best_model_at_end=True,
                metric_for_best_model="f1",
                greater_is_better=True,
                report_to=None,
                save_total_limit=2,
                dataloader_pin_memory=False,
                fp16=torch.cuda.is_available(),
                gradient_checkpointing=True,     # Memory efficient
            )
        else:
            # Full fine-tuning settings - more conservative
            training_args = TrainingArguments(
                output_dir=output_dir,
                num_train_epochs=3,
                per_device_train_batch_size=16,  # Lower batch size for full fine-tuning
                per_device_eval_batch_size=32,
                learning_rate=2e-5,              # Lower learning rate for full fine-tuning
                warmup_steps=500,
                weight_decay=0.01,
                logging_dir='./logs',
                logging_steps=100,
                evaluation_strategy="steps",
                eval_steps=500,
                save_strategy="steps",
                save_steps=500,
                load_best_model_at_end=True,
                metric_for_best_model="f1",
                greater_is_better=True,
                report_to=None,
                save_total_limit=2,
                dataloader_pin_memory=False,
                fp16=torch.cuda.is_available(),
            )

        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=self.compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )

        # Train the model
        self.trainer.train()

        return self.trainer

    def evaluate_model(self, test_dataset, test_labels):
        """Evaluate the model and return detailed metrics"""
        print("Evaluating model...")

        # Get predictions
        predictions = self.trainer.predict(test_dataset)
        y_pred = np.argmax(predictions.predictions, axis=1)
        y_true = test_labels

        # Calculate metrics
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
        accuracy = accuracy_score(y_true, y_pred)

        print(f"\nTest Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")

        # Detailed classification report
        print(f"\nDetailed Classification Report:")
        target_names = self.label_encoder.classes_
        print(classification_report(y_true, y_pred, target_names=target_names))

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'y_true': y_true,
            'y_pred': y_pred,
            'predictions': predictions
        }

    def cross_validate(self, df, cv_folds=5):
        """Perform k-fold cross-validation"""
        print(f"Performing {cv_folds}-fold cross-validation...")

        X = df['enhanced_text'].values
        y = self.label_encoder.fit_transform(df['sentiment'])

        skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

        cv_results = {
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        }

        for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
            print(f"\nFold {fold + 1}/{cv_folds}")

            # Split data
            X_train_fold, X_val_fold = X[train_idx], X[val_idx]
            y_train_fold, y_val_fold = y[train_idx], y[val_idx]

            # Create datasets
            train_dataset = ReviewDataset(X_train_fold, y_train_fold, self.tokenizer)
            val_dataset = ReviewDataset(X_val_fold, y_val_fold, self.tokenizer)

            # Re-initialize model for each fold
            base_model = AutoModelForSequenceClassification.from_pretrained(
                self.model_name,
                num_labels=len(self.label_encoder.classes_)
            )

            if self.use_lora:
                fold_model = get_peft_model(base_model, self.lora_config)
            else:
                fold_model = base_model

            fold_model.to(self.device)

            # Train model
            trainer = self.train_fold_model(fold_model, train_dataset, val_dataset, fold)

            # Evaluate
            predictions = trainer.predict(val_dataset)
            y_pred_fold = np.argmax(predictions.predictions, axis=1)

            # Calculate metrics
            accuracy = accuracy_score(y_val_fold, y_pred_fold)
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_val_fold, y_pred_fold, average='weighted'
            )

            cv_results['accuracy'].append(accuracy)
            cv_results['precision'].append(precision)
            cv_results['recall'].append(recall)
            cv_results['f1'].append(f1)

            print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

            # Clean up
            del fold_model, trainer
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

        # Print cross-validation results
        print(f"\nCross-Validation Results ({cv_folds} folds):")
        for metric, scores in cv_results.items():
            mean_score = np.mean(scores)
            std_score = np.std(scores)
            print(f"{metric.title()}: {mean_score:.4f} (+/- {std_score * 2:.4f})")

        return cv_results

    def train_fold_model(self, model, train_dataset, val_dataset, fold):
        """Train model for a specific fold"""
        if self.use_lora:
            training_args = TrainingArguments(
                output_dir=f'./results_fold_{fold}',
                num_train_epochs=3,
                per_device_train_batch_size=32,
                per_device_eval_batch_size=64,
                learning_rate=1e-4,
                warmup_steps=50,
                weight_decay=0.01,
                logging_steps=25,
                evaluation_strategy="no",
                save_strategy="no",
                report_to=None,
                fp16=torch.cuda.is_available(),
                gradient_checkpointing=True,
            )
        else:
            training_args = TrainingArguments(
                output_dir=f'./results_fold_{fold}',
                num_train_epochs=2,
                per_device_train_batch_size=16,
                per_device_eval_batch_size=32,
                learning_rate=2e-5,
                warmup_steps=100,
                weight_decay=0.01,
                logging_steps=50,
                evaluation_strategy="no",
                save_strategy="no",
                report_to=None,
                fp16=torch.cuda.is_available(),
            )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=self.compute_metrics,
        )

        trainer.train()
        return trainer

    def plot_confusion_matrix(self, y_true, y_pred, title="Confusion Matrix"):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(10, 8))

        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=self.label_encoder.classes_,
                   yticklabels=self.label_encoder.classes_,
                   cbar_kws={'label': 'Count'})

        method = "LoRA" if self.use_lora else "Full Fine-tuning"
        plt.title(f'{title}\nModel: {self.model_name} ({method})', fontsize=16, fontweight='bold')
        plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
        plt.ylabel('True Label', fontsize=12, fontweight='bold')

        # Add accuracy to the plot
        accuracy = accuracy_score(y_true, y_pred) * 100
        plt.figtext(0.02, 0.02, f'Overall Accuracy: {accuracy:.2f}%',
                   fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))

        plt.tight_layout()
        plt.show()

        # Print per-class metrics
        print(f"\nPer-class metrics:")
        precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None)

        for i, class_name in enumerate(self.label_encoder.classes_):
            print(f"{class_name}:")
            print(f"  Precision: {precision[i]:.4f}")
            print(f"  Recall: {recall[i]:.4f}")
            print(f"  F1-Score: {f1[i]:.4f}")
            print(f"  Support: {support[i]}")

    def plot_training_history(self):
        """Plot training history if available"""
        if self.trainer and hasattr(self.trainer.state, 'log_history'):
            history = self.trainer.state.log_history

            train_loss = []
            eval_loss = []
            eval_f1 = []

            for log in history:
                if 'loss' in log:
                    train_loss.append(log['loss'])
                if 'eval_loss' in log:
                    eval_loss.append(log['eval_loss'])
                if 'eval_f1' in log:
                    eval_f1.append(log['eval_f1'])

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

            method = "LoRA" if self.use_lora else "Full Fine-tuning"

            # Loss plot
            ax1.plot(train_loss, label='Training Loss', color='blue')
            if eval_loss:
                ax1.plot(eval_loss, label='Validation Loss', color='red')
            ax1.set_title(f'Training History - {method}\nModel: {self.model_name}')
            ax1.set_xlabel('Steps')
            ax1.set_ylabel('Loss')
            ax1.legend()
            ax1.grid(True)

            # F1 plot
            if eval_f1:
                ax2.plot(eval_f1, label='Validation F1', color='green')
                ax2.set_title(f'Validation F1 Score - {method}')
                ax2.set_xlabel('Steps')
                ax2.set_ylabel('F1 Score')
                ax2.legend()
                ax2.grid(True)

            plt.tight_layout()
            plt.show()

    def save_model(self, output_dir="./final_model"):
        """Save the trained model"""
        if self.use_lora:
            self.peft_model.save_pretrained(output_dir)
            print(f"LoRA model saved to {output_dir}")
        else:
            self.model.save_pretrained(output_dir)
            print(f"Full model saved to {output_dir}")

        self.tokenizer.save_pretrained(output_dir)

    def run_pipeline(self, include_cv=False):
        """Run the complete ABSA pipeline"""
        # Load and preprocess data
        df = self.load_and_preprocess_data()

        # Extract aspects and enhance text
        df = self.extract_aspects_and_enhance_text(df)

        # Encode labels
        y = self.label_encoder.fit_transform(df['sentiment'])
        X = df['enhanced_text'].values

        # Prepare model and tokenizer
        self.prepare_model_and_tokenizer(len(self.label_encoder.classes_))

        # Train-test split (80-20)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        # Further split training data for validation
        X_train_final, X_val, y_train_final, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
        )

        print(f"Final training set size: {len(X_train_final)}")
        print(f"Validation set size: {len(X_val)}")
        print(f"Test set size: {len(X_test)}")

        # Create datasets
        train_dataset, val_dataset, test_dataset = self.create_datasets(
            X_train_final, y_train_final, X_val, y_val, X_test, y_test
        )

        # Train model
        self.train_model(train_dataset, val_dataset)

        # Plot training history
        self.plot_training_history()

        # Evaluate on test set
        test_results = self.evaluate_model(test_dataset, y_test)

        # Plot confusion matrix
        self.plot_confusion_matrix(
            test_results['y_true'],
            test_results['y_pred'],
            f"Test Set Confusion Matrix"
        )

        # Cross-validation (optional)
        cv_results = None
        if include_cv:
            cv_results = self.cross_validate(df, cv_folds=3)  # Reduced folds for LoRA

        # Save model
        self.save_model()

        return {
            'test_results': test_results,
            'cv_results': cv_results,
            'model': self.model,
            'tokenizer': self.tokenizer
        }

def compare_lora_vs_full_finetuning():
    """Compare LoRA vs Full Fine-tuning"""
    print("Comparing LoRA vs Full Fine-tuning...")

    results_comparison = {}

    # Test LoRA
    print(f"\n{'='*60}")
    print(f"TESTING LoRA FINE-TUNING")
    print(f"{'='*60}")

    pipeline_lora = ABSALLMPipeline(model_name='distilbert-base-uncased', use_lora=True)
    results_lora = pipeline_lora.run_pipeline(include_cv=False)

    results_comparison['LoRA'] = {
        'accuracy': results_lora['test_results']['accuracy'],
        'precision': results_lora['test_results']['precision'],
        'recall': results_lora['test_results']['recall'],
        'f1': results_lora['test_results']['f1']
    }

    # Clear memory
    del pipeline_lora
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # Test Full Fine-tuning
    print(f"\n{'='*60}")
    print(f"TESTING FULL FINE-TUNING")
    print(f"{'='*60}")

    pipeline_full = ABSALLMPipeline(model_name='distilbert-base-uncased', use_lora=False)
    results_full = pipeline_full.run_pipeline(include_cv=False)

    results_comparison['Full Fine-tuning'] = {
        'accuracy': results_full['test_results']['accuracy'],
        'precision': results_full['test_results']['precision'],
        'recall': results_full['test_results']['recall'],
        'f1': results_full['test_results']['f1']
    }

    # Print comparison
    print(f"\n{'='*60}")
    print("LoRA vs FULL FINE-TUNING COMPARISON")
    print(f"{'='*60}")

    df_comparison = pd.DataFrame(results_comparison).T
    print(df_comparison.round(4))

    return results_comparison

# Main execution
if __name__ == "__main__":
    # Option 1: Run LoRA fine-tuning (recommended for T4)
    print("Running ABSA Pipeline with LoRA Fine-tuning...")
    pipeline = ABSALLMPipeline(model_name='distilbert-base-uncased', use_lora=True)
    results = pipeline.run_pipeline(include_cv=True)

    print(f"\n{'='*50}")
    print("LORA PIPELINE COMPLETED!")
    print(f"{'='*50}")
    print(f"Final Test Results:")
    print(f"Accuracy: {results['test_results']['accuracy']:.4f}")
    print(f"Precision: {results['test_results']['precision']:.4f}")
    print(f"Recall: {results['test_results']['recall']:.4f}")
    print(f"F1-Score: {results['test_results']['f1']:.4f}")

    # Option 2: Compare LoRA vs Full Fine-tuning
    # comparison_results = compare_lora_vs_full_finetuning()

    # Option 3: Run full fine-tuning
    # pipeline_full = ABSALLMPipeline(model_name='distilbert-base-uncased', use_lora=False)
    # results_full = pipeline_full.run_pipeline(include_cv=False)

In [ ]:
from pandas_gbq import to_gbq

PROJECT_ID = 'able-balm-454718-n8'  # your GCP project ID
DATASET = 'absa_outputs'  # or any dataset name you like (will be created if it doesn't exist)

# Helper to upload DataFrame
def upload_to_bigquery(df, table, project_id=PROJECT_ID, dataset=DATASET):
    full_table = f"{dataset}.{table}"
    df.to_gbq(full_table, project_id=project_id, if_exists='replace')
    print(f"Uploaded {table} to BigQuery dataset {dataset}.")

# Export train/test data
upload_to_bigquery(train_data, 'train_data')
upload_to_bigquery(test_data, 'test_data')

# Export metrics
upload_to_bigquery(summary_df, 'test_metrics')

# Export recommendations and mixed reviews
upload_to_bigquery(rec_df, 'business_recommendations')
upload_to_bigquery(mixed_df, 'mixed_reviews')
